In [22]:
from hearing_assistive import *
import numpy as np

%matplotlib inline

# Generate Dataset and Targetset

In [23]:
wave = read_wave('Sounds/school_fire_alarm.wav')
wave.normalize()

# partition the wave into segments of 2.4 seconds
i = 0
j = int(2.4 * wave.framerate)  # frames per segment
step = int(1.2 * wave.framerate)   # 50% overlap

seg_map = []
while j < len(wave.ys):
    segment = wave.slice(i, j)
    seg_map.append(segment)
    
    i += step
    j += step

In [24]:
data_set = []
target_set = []

# for i in range(len(seg_map)):
#     signal = seg_map[i]
#     spectrogram = signal.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
#     feature = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
#     data_set.append(feature)
#     target_set.append(2)


signal = seg_map[0]
spectrogram = signal.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
feature = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
data_set.append(feature)
target_set.append(2)

In [25]:
# add 10 dummy sets to the data set
for i in range(10):
    dummy_set = np.zeros((119, 13))
    data_set.append(dummy_set)
    target_set.append(0)

In [26]:
# convert these arrays into numpy arrays
data_set = np.asarray(data_set)
target_set = np.asarray(target_set)

print len(data_set)
print data_set.shape

11
(11, 119, 13)


# Create and Train SVM Classifier

In [27]:
from sklearn import linear_model

In [28]:
nsamples, nsegments, coef = data_set.shape
data_set = data_set.reshape((nsamples, nsegments * coef))

print len(data_set)
print data_set.shape
print len(target_set)
print target_set.shape

11
(11, 1547)
11
(11,)


In [29]:
clf = linear_model.SGDClassifier()

clf.fit(data_set, target_set)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

# Create Testing Set and Test SVM

In [9]:
# create testing set
test_set = []

signal = seg_map[10]
spectrogram = signal.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
feature = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
test_set.append(feature)

In [10]:
feature = np.zeros((119, 13))
test_set.append(feature)

In [11]:
signal = seg_map[15]
spectrogram = signal.make_spectrogram(seg_length=1764)  # 1764 = 40ms * 44100fps = samples per 40ms
feature = spectrogram.mfcc()   # use default parameter, only include coefficients 2-14 (13 coefficients)
test_set.append(feature)

In [12]:
test_set = np.asarray(test_set)
nsamples, nsegments, coef = test_set.shape
test_set = test_set.reshape((nsamples, nsegments * coef))

In [13]:
clf.predict(test_set)

array([2, 0, 2])

# Introduce New Data

In [14]:
wave = read_wave('Sounds/police_siren.wav')
print wave.framerate

i = 0
j = int(2.4 * wave.framerate)  # frames per segment
step = int(1.2 * wave.framerate)   # 50% overlap

seg_map = []
while j < len(wave.ys):
    segment = wave.slice(i, j)
    seg_map.append(segment)
    
    i += step
    j += step

22050


In [15]:
data_set = []
target_set = []

for i in range(len(seg_map)):
    signal = seg_map[i]
    spectrogram = signal.make_spectrogram(seg_length=882)  # 882 = 40ms * 22050fps = samples per 40ms
    feature = spectrogram.mfcc(blocksize=442)   # use default parameter, only include coefficients 2-14 (13 coefficients)
    data_set.append(feature)
    target_set.append(3)

In [16]:
data_set = np.asarray(data_set)
target_set = np.asarray(target_set)

In [17]:
nsamples, nsegments, coef = data_set.shape
data_set = data_set.reshape((nsamples, nsegments * coef))

In [18]:
from copy import copy

clf2 = copy(clf)

clf2.partial_fit(data_set, target_set)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

In [19]:
new_test_set = []
new_test_set.append(test_set[0])
new_test_set.append(test_set[1])
new_test_set.append(test_set[2])

signal = seg_map[5]
spectrogram = signal.make_spectrogram(seg_length=882)  # 882 = 40ms * 22050fps = samples per 40ms
feature = spectrogram.mfcc(blocksize=442)   # use default parameter, only include coefficients 2-14 (13 coefficients)
nsegments, coef = feature.shape
feature = feature.reshape(nsegments * coef)
new_test_set.append(feature)

feature = np.zeros((119, 13))
feature += 100
nsegments, coef = feature.shape
feature = feature.reshape(nsegments * coef)
new_test_set.append(feature)


In [20]:
new_test_set = np.asarray(new_test_set)
clf2.predict(new_test_set)

array([2, 0, 2, 0, 2])